# Testing

In [1]:
import os
from configparser import ConfigParser

import pyspark
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql import SparkSession
import boto3
from botocore.config import Config
#from boto.s3.connection import S3Connection

In [8]:
#PROFILE_ID="119657064844_Admin_UNRESTRICTED"
PROFILE_ID='default'
config_object = ConfigParser()
config_object.read("/home/jovyan/.aws/credentials")
profile_info = config_object[PROFILE_ID]

In [9]:
ACCESS_KEY = profile_info.get('aws_access_key_id')
SECRET_KEY = profile_info.get('aws_secret_access_key')
AWS_SESSION_TOKEN = profile_info.get('aws_session_token')

In [10]:
#!cat ~/.aws/credentials

    %env AWS_ACCESS_KEY_ID={ACCESS_KEY}
    %env AWS_SECRET_ACCESS_KEY=+{SECRET_KEY}
    %env AWS_SESSION_TOKEN={AWS_SESSION_TOKEN}

    !echo $AWS_ACCESS_KEY_ID

## Connect to S3 client

In [11]:
s3 = boto3.client(
    's3',
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET_KEY,
    aws_session_token=AWS_SESSION_TOKEN
)


Test successful connection

In [13]:
response = s3.list_buckets()
for bucket in response['Buckets'][:3]:
    print(bucket['Name'])

aws-athena-query-results-916122220265-us-east-1
aws-logs-916122220265-us-east-1
cdn-origin-us-east-1-qa-ackbar-teal


### Download Test File Locally

    df_visitors = spark.read.parquet('s3://think-training-data-us-east-1-prod-teal/advanceautoparts/main/all-visitors/')
    df_visitors = df_visitors.where("visit_day >= '2020-08-01'") 

    (
        df_visitors
        .write
        .option('compression', 'snappy')
        .mode("overwrite")
        .partitionBy("visit_day")
        .parquet("s3://jvs-fraud-research/all-visitors/advanceautoparts/main/")
    )

The following is found in the Production - Public Cloud environment and is a good start to testing:  

        s3://think-training-data-us-east-1-prod-teal/advanceautoparts/main/all-visitors/

After running the move_data Glue job the data can be found:
        
        s3://jvs-fraud-research/all-visitors/advanceautoparts/main/
        
This data was queried only where `visit_day >= '2020-08-01'`

In [8]:
# Download the file from S3
bucket = "think-training-data-us-east-1-prod-teal/"
filepath = "advanceautoparts/main/all-visitors/visit_day=2019-07-03/part-00000-0030db36-139c-4699-a5c8-48696c54ccb3.c000.snappy.parquet"
local_filename = 'test.snappy.parquet'
s3.download_file(bucket, filepath, local_filename)

## Using Spark directly to connect to S3

In [9]:
os.environ['PYSPARK_SUBMIT_ARGS'] = "--packages=com.amazonaws:aws-java-sdk-bundle:1.11.271,org.apache.hadoop:hadoop-aws:3.1.2 pyspark-shell"
conf = (
    SparkConf()
    .set("spark.hadoop.fs.s3a.path.style.access", True)
    
    .set("spark.hadoop.fs.s3a.aws.credentials.provider","org.apache.hadoop.fs.s3a.TemporaryAWSCredentialsProvider")
    .set("spark.hadoop.fs.s3a.access.key", profile_info.get('aws_access_key_id'))
    .set("spark.hadoop.fs.s3a.secret.key", profile_info.get('aws_secret_access_key'))
    .set("spark.hadoop.fs.s3a.session.token", profile_info.get('aws_session_token'))
    
    .set("spark.hadoop.fs.s3a.multipart.size", 104857600)
    
    .set("spark.hadoop.fs.s3a.endpoint", f"s3-{profile_info.get('region')}.amazonaws.com")
    .set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .set("com.amazonaws.services.s3.enableV4", True)
    .set("spark.driver.extraJavaOptions", "-Dcom.amazonaws.services.s3.enableV4=true")
)
sc = SparkContext(conf=conf).getOrCreate()
sqlContext = SQLContext(sc)

In [10]:
!echo $PYSPARK_SUBMIT_ARGS

--packages=com.amazonaws:aws-java-sdk-bundle:1.11.271,org.apache.hadoop:hadoop-aws:3.1.2 pyspark-shell


Test That sc works

In [11]:
rdd = sc.parallelize(range(1000))
rdd.takeSample(False, 5)

[169, 228, 67, 337, 437]

In [12]:
example_s3_parquet="s3a://tealium-ml-research/joo/binary_class/aa_14_30days_20190801_20200510/part-00000-de89b881-1515-4cb2-bc05-7535e6325736-c000.snappy.parquet" 


Confirm that sqlcontext can read parquet files locally

In [13]:
df = sqlContext.read.parquet(local_filename)

In [14]:
df.columns[:10]

['visitor_id',
 'visit_day',
 'visit_dates_10',
 'visit_properties_5281',
 'visitor_badges_14263',
 'visit_metrics_80',
 'visit_metrics_14447',
 'visitor_metrics_14301',
 'visitor_badges_5359',
 'visitor_badges_7976']

In [15]:
#import findspark
#findspark.init()
#from pyspark.sql import SparkSession

In [16]:
df = sqlContext.read.parquet(example_s3_parquet)

Py4JJavaError: An error occurred while calling o73.parquet.
: java.lang.IllegalArgumentException
	at java.base/java.util.concurrent.ThreadPoolExecutor.<init>(ThreadPoolExecutor.java:1293)
	at java.base/java.util.concurrent.ThreadPoolExecutor.<init>(ThreadPoolExecutor.java:1215)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.initialize(S3AFileSystem.java:280)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3303)
	at org.apache.hadoop.fs.FileSystem.access$200(FileSystem.java:124)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3352)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3320)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:479)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:361)
	at org.apache.spark.sql.execution.streaming.FileStreamSink$.hasMetadata(FileStreamSink.scala:46)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:366)
	at org.apache.spark.sql.DataFrameReader.l

Alternative configuration setup
    
    spark = (
        SparkSession
        .builder
        .appName("Data Science Project")
        #.config("spark.some.config.option", "some-value")
        .getOrCreate()
    )
    
    sc = spark.sparkContext
    spark._jsc.hadoopConfiguration().set("fs.s3a.access.key", ACCESS_KEY)
    spark._jsc.hadoopConfiguration().set("fs.s3a.secret.key", SECRET_KEY)
    spark._jsc.hadoopConfiguration().set("fs")
    #spark._jsc.hadoopConfiguration().set("fs.s3a.")
    spark._jsc.hadoopConfiguration().set("fs.s3a.impl","org.apache.hadoop.fs.s3a.S3AFileSystem")

In [ ]:
raw_data_ex = "s3://think-training-data-us-east-1-prod-teal"
downsampled_data = "s3://tealium-ml-research/joo/"
adv_auto_parts_data = "s3://think-training-data-us-east-1-prod-teal/advanceautoparts/main/all-visitors/"